In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as colors

In [2]:
# K Means clusters
k_means_df = pd.read_csv('k_means_ratings.csv')
# Weighted Ratings df
weighted_df = pd.read_csv('ratings_weighted.csv')
# Import Movies
movies_df = pd.read_csv('movies.csv')

In [3]:
# userid = 1
userid = 215
# userid = 1 - normal cluster
# userid = 215 - Cluster 1
# 1578 - Cluster 1

In [4]:
id_single_df = k_means_df.loc[k_means_df['userId'] == userid]
id_single_df

,userId,movieId,rating,genres,KMeans
3890,215,899,1.0,Comedy|Musical|Romance,1
4319,215,1088,1.0,Drama|Musical|Romance,12
7035,215,2011,3.0,Adventure|Comedy|Sci-Fi,5
41455,215,356,5.0,Comedy|Drama|Romance|War,15
44761,215,364,2.0,Adventure|Animation|Children|Drama|Musical|IMAX,12
...,...,...,...,...,...
900703,215,4783,5.0,Documentary,15
900733,215,5303,1.0,Comedy|Romance,1
900813,215,5694,3.0,Comedy|Drama|Musical,19
900831,215,5927,2.0,Comedy|Drama|Musical|Romance,12


In [5]:
cluster_number = id_single_df['KMeans'].value_counts().idxmax()
cluster_number

1

In [6]:
# k_means_user_df = id_single_df.loc[(id_single_df["KMeans"] == cluster_number) & (id_single_df["userId"] == userid)]
# k_means_user_df

In [7]:
k_means_cluster_df = k_means_df.loc[(k_means_df["KMeans"] == cluster_number)]
k_means_cluster_df

,userId,movieId,rating,genres,KMeans
3890,215,899,1.0,Comedy|Musical|Romance,1
3915,610,899,1.0,Comedy|Musical|Romance,1
3968,1384,899,1.0,Comedy|Musical|Romance,1
3990,1784,899,0.5,Comedy|Musical|Romance,1
3996,1848,899,2.0,Comedy|Musical|Romance,1
...,...,...,...,...,...
999950,6558,6353,0.5,Comedy|Musical,1
999952,6558,6937,0.5,Musical|Romance,1
999955,6558,8954,0.5,Documentary,1
999963,6591,3642,2.0,Western,1


In [8]:
good_rows = k_means_cluster_df[k_means_cluster_df["rating"] >= 4]
bad_rows = k_means_cluster_df[k_means_cluster_df["rating"] < 4]

In [9]:
good_rows

,userId,movieId,rating,genres,KMeans


In [10]:
bad_rows

,userId,movieId,rating,genres,KMeans
3890,215,899,1.0,Comedy|Musical|Romance,1
3915,610,899,1.0,Comedy|Musical|Romance,1
3968,1384,899,1.0,Comedy|Musical|Romance,1
3990,1784,899,0.5,Comedy|Musical|Romance,1
3996,1848,899,2.0,Comedy|Musical|Romance,1
...,...,...,...,...,...
999950,6558,6353,0.5,Comedy|Musical,1
999952,6558,6937,0.5,Musical|Romance,1
999955,6558,8954,0.5,Documentary,1
999963,6591,3642,2.0,Western,1


In [11]:
# SELECT GOOD MOVIES THAT HAVE NOT BEEN RATED BY THIS USER
good_rows = good_rows[good_rows["userId"] != userid]
good_rows = good_rows.drop_duplicates(subset=['movieId'])

In [12]:
recommended_df = pd.DataFrame()

if good_rows.shape[0] >= 15:
    # GET THE WEIGHTED RATINGS FOR THE ABOVE MOVIES
    new_good_rows = pd.merge(good_rows, weighted_df[["movieId", "Weighted_Rating_W"]], on="movieId")

    # GET THE MOVIE TITLES AS WELL
    new_good_rows = pd.merge(new_good_rows, movies_df[["movieId", "title"]], on="movieId")

    # SHOW THE TOP 15 MOVIES - RECOMMENDED MOVIES FOR THIS USER (FROM THIS CLUSTER)
    recommended_df = new_good_rows.sort_values('Weighted_Rating_W', ascending=False).head(15)


In [13]:
# IF NEGATIVE RATINGS ARE MORE FREQUENT

In [14]:
if good_rows.shape[0] < 15:

    # SELECT MOVIES THAT HAVE NOT BEEN RATED BY THIS USER
    bad_rows = bad_rows[bad_rows["userId"] != userid]
    bad_rows = bad_rows.drop_duplicates(subset=['movieId'])

    # GET 3 LEAST FREQUENT GENRES - THOSE WILL BE THE LEAST DISLIKED GENRES
    all_genres_repeated = list(bad_rows['genres'].str.split('|', expand=True).stack())
    all_genres_repeated_df = pd.DataFrame(all_genres_repeated, columns=["genres"])
    all_genres_repeated_df = all_genres_repeated_df.groupby(['genres'])['genres'].count().reset_index(name='counts').sort_values('counts', ascending=False)
    least_frequest_disliked_df = all_genres_repeated_df[-3:]
    print(least_frequest_disliked_df)

    # SEARCH FOR THOSE GENRES IN ALL THE MOVIES (INCLUDING OUTSIDE THE CLUSTER)
    movies_pool = movies_df[movies_df['genres'].str.contains(least_frequest_disliked_df.iloc[0][0] + '|' + least_frequest_disliked_df.iloc[1][0] + '|' + least_frequest_disliked_df.iloc[2][0])]
    recommended_df = pd.merge(movies_pool, weighted_df[["movieId", "Weighted_Rating_W"]], on="movieId")
    
    # RECOMMEND THE TOP 15 MOVIES
    recommended_df = recommended_df.sort_values('Weighted_Rating_W', ascending=False).head(15)

       genres  counts
12       IMAX      52
8       Drama      24
10  Film-Noir       6


In [15]:
recommended_df[['movieId','title', 'Weighted_Rating_W', 'genres']]

,movieId,title,Weighted_Rating_W,genres
151,318,"Shawshank Redemption, The (1994)",4.410568,Crime|Drama
343,858,"Godfather, The (1972)",4.319984,Crime|Drama
480,1221,"Godfather: Part II, The (1974)",4.255421,Crime|Drama
249,527,Schindler's List (1993),4.244027,Drama|War
3983,170705,Band of Brothers (2001),4.240781,Action|Drama|War
774,2019,Seven Samurai (Shichinin no samurai) (1954),4.238787,Action|Adventure|Drama
468,1203,12 Angry Men (1957),4.230215,Drama
1079,2959,Fight Club (1999),4.224719,Action|Crime|Drama|Thriller
467,1193,One Flew Over the Cuckoo's Nest (1975),4.212868,Drama
359,912,Casablanca (1942),4.198890,Drama|Romance
